# Cuerda
Una cuerda de densidad lineal ρ está fija en x = 0. La cuerda tiene longitud L. El borde en x = L puede estar fijo, o bien, deslizarse sobre una corredera (sin fricción), según se muestra en la figura.

![Figura 1](TP1_Esquema.png)

La condición de borde para el desplazamiento transversal $\psi$ de la cuerda fija-fija es

\begin{equation}\left\{\begin{array}{ll}
\psi |_{x=0}=0\\
\psi |_{x=L}=0
\end{array}\right.\label{eq1}\tag{1}\end{equation}

Y la condicion de borda para el desplazamiento transversal $psi$ de la cuerda fija-libre es

\begin{equation}\left\{\begin{array}{ll}
\psi |_{x=0}=0\\
\frac{\partial\psi}{\partial x} |_{x=L}=0
\end{array}\right.\label{eq2}\tag{2}\end{equation}

El modelo de pequeña amplitud transversal de la cuerda cuando está tensionada es el siguiente

$$f(x)=T_{0}\frac{\partial^{2}\psi}{\partial x^{2}}\implies\rho\frac{\partial^{2}\psi}{\partial t^{2}}=T_{0}\frac{\partial^{2}\psi}{\partial x^{2}}\label{eq3}\tag{3}$$

donde $f(x)$ es la fuerza (tangencial) por unidad de longitud actuando en la cuerda, y $T_0$ es la tensión de la misma. $\rho$ es su densidad lineal.

### Inciso a)

Considere un movimiento armónico para cada punto de la cuerda $\psi=-\omega^{2}\psi$. De-
termine las frecuencias naturales de la cuerda bajo la condición de borde “fijo” y de “corredera” en el punto $x=L$ (recuerde que $\psi(0)=0$). Para ello, aproxime la derivada segunda en la ec. (\ref{eq3}) del siguiente modo

$$\frac{\partial^{2}\psi}{\partial x^{2}}\approx\frac{1}{h^{2}}[\psi_{n+1}(t)-2\psi_n(t)+\psi_{n-1}(t)]\label{eq4}\tag{4}$$


donde los desplazamientos $\psi_0, \psi_1, ..., \psi_{N+1}$ corresponden a puntos equidistantes de la cuerda, separados por una distancia $h=L/(N+1)$. El desplazamiento $\psi_0$ corresponde al borde $x=0$ y el desplazamiento $\psi_{N+1}$ corresponde al borde $x=L$.

Exprese la ec. (\ref{eq3}) como un sistema lineal. Imponga las condiciones de borde sobre $\psi_0$ y $\psi_{N+1}$ de manera que el sistema quede expresado como función de los desplazamientos ($\psi_1, ..., \psi_N$).

Identifique los auto-valores $\mu^{2}$ asociados con los auto-vectores columna ($\psi_1, ..., \psi_N$). Relacione los auto-valores $\mu^{2}$ con la frecuencia natural $\omega$ y con la longitud de onda asociada $\lambda=2\pi c/\omega$ ($c^{2}=T_0/\rho$). Esta última resulta más práctica para visualizar el fenómeno.

Calcule las longitudes de onda $\lambda$ compatibles con ambas condiciones de borde. Para ello, utilice un código computacional capaz de obtener los auto-valores $\mu^{2}$ y sus lon- gitudes asociadas. Si decide asistirse por Python, primero debe construir las matrices de coeficientes asociadas al sistema (2), y luego obtener los autovalores.

Una forma fácil de construir una matriz tridiagonal (de tamaño $N$) en Python es por medio del siguiente código.

```
a = np.zeros(N)
a[0] = 2 a[1] = -1
first_row = np.array(a) first_col = np.array(a)
B = toeplitz(first_col, first_row)
print(B)
```

Recuerde importar las librerías numpy, scipy y pylab para correrlo. Modifique este código para adaptarlo a sus necesidades.

El cómputo de los auto-valores $\mu^{2}$ (y auto-vectores) se lo puede realizar por medio de las instrucciones.

```
mu, v = la.eig(B)
print(np.sort(mu))
```

donde mu y v son los auto-valores $\mu^{2}$ y auto-vectores, respectivamente. La instrucción np.sort(mu) es para que los auto-valores se impriman en orden ascendente.

### Inciso b)

Analice las longitudes de onda $\lambda$ asociadas a cada auto-valor $\mu$. Para ello estudie la longitud de onda mínima λmin que se obtiene para un número creciente de puntos $N=2,3,4,5,...,50$. Presente los resultados en un gráfico de $\lambda_{min}$ vs. $h=L/(N+1)$ (considere, por simplicidad, $L=1$). Verifique si $\lambda_{min}\approx\pi h$.

Recuerde que la ec. (\ref{eq4}) es equivalente a la siguiente relación de recurrencia.

$$\psi_{n+1}=(2-\mu^{2})\psi_n-\psi_{n-1}\label{eq5}\tag{5}$$

Esta relación indica que sólo es posible obtenerdesplazamientos $\psi_n$ oscilantes para un rango acotado de autovalores $\mu$. Justifique esta afirmación siguiendo razonamientos a su elección. Si lo desea, puede asistirse considerando soluciones de la forma $\psi_n\sim r^{n}$ (para algún r adecuado). Además, argumente (cualitativamente) por qué no se cumple estrictamente $\lambda_{min}=\pi h$ para un número bajo de puntos N (muestrelo en el gráfico)

### Inciso c)

Considere la condición de borde “fija” (si lo desea, puede repetir este mismo análisis para el caso de la “corredera”). Obtenga las longitudes $\lambda$ para $N=2,3,5,7,11$. Analice si las longitudes $\lambda$ para $N=2,3$ son multiplos de las longitudes $\lambda'$ obtenidas para $N=5,7,11$. Es decir, $\lambda=m\lambda'$ donde $m=2,3,...$. Tenga en cuenta que la correspondencia puede ocurrir para más de un valor entero $m$. Escriba las correspondencias halladas.

Concluya a partir de este estudio si la observación de puntos equidistantes permite identificar longitudes de onda “únicas”. Explique qué significa esta “ambigüedad” y qué es el fenómeno de “aliasing” espacial. Si desea, puede ejemplificarlo interpolando sinusoidales de distinta $\lambda$ entre los $N$ puntos.

### Inciso d)

Obtenga la evolución temporal de la cuerda ψn(t) a partir de las condiciones iniciales

\begin{equation}\left\{\begin{array}{ll}
\psi_n(0)=0,2nh/L\qquad\textrm{si}\quad0<nh\leq L/2\\
\psi_n(0)=0,2(1-nh/L)\qquad\textrm{si}\quad L/2<nh\leq L\\
\dot{\psi}_n(0)=0\qquad\textrm{si}\quad0<nh\leq L
\end{array}\right.\label{eq6}\tag{6}\end{equation}

Estas condiciones excitan muchos modos de la cuerda, por lo que debe dejar de lado la relación para un único modo $\ddot{\psi}_n=-\omega^{2}\psi_n$ y reemplazarla por la aproximación

$$\ddot{\psi}_n\approx\frac{1}{\tau^{2}}[\psi_n(t+\tau)-2\psi_n(t)+\psi_n(t-\tau)]\label{eq7}\tag{7}$$

donde $\tau$ es un pequeño intervalo temporal. Si expresa esta aproximación en términos del vector columna $\psi=(\psi_1,...,\psi_n)$ y se iguala con el sistema lineal hallado en el item (a), se arriba al esquema algorítmico de Störmer-Verlet

$$B\Psi(t)\approx\frac{h^{2}}{c^{2}\tau^{2}}[\Psi(t+\tau)-\Psi(t)+\Psi(t-\tau)]\label{eq8}\tag{8}$$

$$\implies\Psi(t+\tau)\approx[2I-\frac{c^{2}\tau^{2}}{h^{2}}B]\Psi(t)-\Psi(t-\tau)\label{eq9}\tag{9}$$

donde $B$ es la matriz generada por medio del código Python en el item (a). $I$ es la matriz identidad (N × N). El esquema de Verlet es fácilmente codificable. En Python se escribe del siguiente modo

```
t = 0.0
r2 = (c**2)*(tau**2)/(h**2)
Psi0 = Psi1

while (t<tmax):
    Psi_new = (2*I-r2*B).dot(Psi1)-Psi0
    Psi0 = Psi1
    Psi1 = Psi_new
    t = t + tau
```

donde $Psi0$ y $Psi1$ corresponden a $\Psi(t-\tau)$ y $\Psi(t)$, respectivamente.

Implemente el esquema de Verlet para las condiciones iniciales detalladas en (\ref{eq6}).

Recuerde que la condición inicial $\dot{\psi}(0)=0$ se puede aproximar a $\psi(\tau)\approx\psi(0)$.
Explique qué suposiciones hace esta aproximación respecto de la aceleración y del equilibrio de fuerzas del sistema.

Simule la evolución del sistema para cinco puntos equidistantes ($N=5$) y un período de tiempo $0<t\leq10$. Suponga $\tau=0,05$ y $c=1,0$. Muestre sus resultados por medio de “fotos” del sistema, o bien, por medio de una animación. Incluya los puntos $\psi_0 y $\psi_{N+1}$ en los gráficos.

Examine en detalle alguno de los puntos (por ejemplo, el punto ubicado en el centro de la cuerda $\psi_3$). Para ello, grafique $\psi_3$ a lo largo del tiempo, es decir, $\psi_3(0)$,$\psi_3(\tau)$,$\psi_3(2\tau)$,$\psi_3(3\tau)$,.... Repita este gráfico para $c=3,33$, $c=3,45$ y $c=3,46$.
Explique los fenómenos de “inestabilidad” que observa. Para ello, tenga en cuenta que la expresión (\ref{eq9}) implica

$$\psi_n(t+\tau)\approx(1-\beta^{2})\psi_n(t)-\psi_n(t-\tau)+\beta^{2}[\psi_{n+1}(t)-\psi_{n-1}(t)]\label{eq10}\tag{10}$$

donde $\beta=c\pi/h$. Realice un análisis similar al del item (b) para saber por qué sólo es posible obtener desplazaminentos oscilantes acotados si $\beta^{2}\leq1$. Esta condición se conoce como condición de Courant.

## a)

SIN PYTHON, LATEXXXXXX

In [15]:
%matplotlib inline

import numpy as np
import scipy as sp
import pylab  as pl
import sympy
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

sympy.init_printing()

In [43]:
N = 5 # cantidad de masas a simular

L = 10       # largo L
T_0 = 4      # tension promedio de la cuerda en reposo
rho = 0.1    # densidad lineal de la cuerda

h = L/(N+1)
c = np.sqrt(T_0/rho)

Escribo la ecuacion (2) como

$$\frac{\partial\psi}{\partial t}\approx-\frac{T_0}{\rho}(2\psi_n-\psi_{n-1}-\psi_{n+1})$$

Que puedo reescribir matricialmente como

$$\ddot{\vec{\psi}}_n+\frac{T_0}{\rho}M\vec{\psi}=0\qquad con \quad M=\begin{bmatrix}2&-1&0&0&...&0\\-1&2&-1&0&...&0\\0&-1&2&-1&...&0\\.&.&&&&.\\.&&.&&&.\\.&&&.&&.\\.&&&&.&.\\0&...&0&-1&2&-1\\0&...&0&0&-1&2\end{bmatrix}$$

Luego, solo resta resolver la ecuacion $det(M-\omega^{2}I)=0$ para obtener los autovalores (que son las frecuencias de los modos normales) y los autovectores (que son las amplitudes de cada masa para cada modo normal)

In [50]:
# crear matriz M
a = sympy.symbols('a')
def assignment(i, j):
  if i == j:
    return 2*a
  if i+1 == j:
    return -a
  if i-1 == j:
    return -a
  return 0

M = sympy.Matrix(N,N,assignment)

sympy.pprint(M)
# encontrar auto-cosas

eigenvals = M.eigenvals()
sympy.pprint(eigenvals)
# eigenvects = M.eigenvects() # OJO con buscar los autovectores porque tarda mil años / no funca con valores de N grandes
# sympy.pprint(M.eigenvects(eigenvects))




⎡2⋅a  -a    0    0    0    0 ⎤
⎢                            ⎥
⎢-a   2⋅a  -a    0    0    0 ⎥
⎢                            ⎥
⎢ 0   -a   2⋅a  -a    0    0 ⎥
⎢                            ⎥
⎢ 0    0   -a   2⋅a  -a    0 ⎥
⎢                            ⎥
⎢ 0    0    0   -a   2⋅a  -a ⎥
⎢                            ⎥
⎣ 0    0    0    0   -a   2⋅a⎦
⎧                                                                             
⎪                                                                             
⎪  ⎛                     _____________                                   ⎞    
⎪  ⎜5   ⎛  1   √3⋅ⅈ⎞    ╱ 7    7⋅√3⋅ⅈ                   7                ⎟    
⎨a⋅⎜─ + ⎜- ─ - ────⎟⋅3 ╱  ── + ──────  + ────────────────────────────────⎟: 1,
⎪  ⎜3   ⎝  2    2  ⎠ ╲╱   54     18                         _____________⎟    
⎪  ⎜                                       ⎛  1   √3⋅ⅈ⎞    ╱ 7    7⋅√3⋅ⅈ ⎟    
⎪  ⎜                                     9⋅⎜- ─ - ────⎟⋅3 ╱  ── + ────── ⎟    
⎩  ⎝                       